In [1]:
#set the current directory for reading the .csv files more easily
from google.colab import drive
drive.mount('/content/drive')
import os
os.chdir('/content/drive/My Drive/Colab Notebooks/kaggle')

Mounted at /content/drive


In [2]:
#import the necessary packages
#import the model XGBoost
import numpy as np
from sklearn import datasets
from sklearn import metrics
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib as mpl
from sklearn import neighbors
from sklearn.utils import shuffle
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split, cross_val_predict
from sklearn.model_selection import cross_val_score, GridSearchCV
import xgboost as xgb
from xgboost import XGBRegressor

In [3]:
#read the cleaned data
import_train_data = pd.read_csv("trainingdata_cleaned-1.csv")
import_test_data = pd.read_csv("testingdata_cleaned-1.csv")
submission = pd.read_csv("sampleSubmission.csv")
import_test_data

,id,hour,month,day,year,weekday,holiday,tempC,visibility,winddirDegree,windspeedKmph,humidity,cloudcover,WindChillC,speed
0,0,2,1,1,2018,0,1,19,10,65,12,63,23,18,0
1,1,5,1,1,2018,0,1,19,10,65,12,63,23,18,0
2,2,7,1,1,2018,0,1,19,10,65,12,63,23,18,0
3,3,8,1,1,2018,0,1,19,10,65,12,63,23,18,0
4,4,10,1,1,2018,0,1,19,10,65,12,63,23,18,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3499,3499,17,12,31,2018,0,0,12,10,138,18,69,79,10,0
3500,3500,19,12,31,2018,0,0,12,10,138,18,69,79,10,0
3501,3501,21,12,31,2018,0,0,12,10,138,18,69,79,10,0
3502,3502,22,12,31,2018,0,0,12,10,138,18,69,79,10,0


In [4]:
input_features=import_train_data[['holiday','hour','month','day','year','weekday','tempC','visibility','winddirDegree','windspeedKmph','humidity','cloudcover','WindChillC']]
prediction_input=import_test_data[['holiday','hour','month','day','year','weekday','tempC','visibility','winddirDegree','windspeedKmph','humidity','cloudcover','WindChillC']]
input_target=import_train_data[['speed']]

In [5]:
#divide the test data
X_train, X_test, y_train, y_test = train_test_split(input_features, input_target, test_size=0.01, random_state=1)

In [6]:
model = xgb.XGBRegressor(base_score=0.7, booster='gbtree', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=1, gamma=0.85,
             importance_type='gain', learning_rate=0.08, max_delta_step=0,
             max_depth=6, min_child_weight=1, missing=None, n_estimators=580,
             n_jobs=1, nthread=None, objective='reg:linear', random_state=0,
             reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
             silent=None, subsample=0.7, verbosity=1)
model.fit(X_train, y_train)

[12:59:49] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


XGBRegressor(base_score=0.7, booster='gbtree', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=1, gamma=0.85,
             importance_type='gain', learning_rate=0.08, max_delta_step=0,
             max_depth=6, min_child_weight=1, missing=None, n_estimators=580,
             n_jobs=1, nthread=None, objective='reg:linear', random_state=0,
             reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
             silent=None, subsample=0.7, verbosity=1)

In [7]:
y_prediction = model.predict(X_test)
#output the MSE
MSE = metrics.mean_squared_error(y_test, y_prediction)
print('MSE:',MSE)

MSE: 7.949155599455978


In [8]:
#implement the prediction for testing dataset
submission_speed = model.predict(prediction_input)
submission_speed = pd.DataFrame(submission_speed)
print(submission_speed)

              0
0     48.205376
1     48.265476
2     39.312901
3     30.481630
4     39.188168
...         ...
3499  12.140996
3500  24.600855
3501  48.416721
3502  41.432034
3503  44.516960

[3504 rows x 1 columns]


In [9]:
submission["speed"] = submission_speed
print(submission)

        id      speed
0        0  48.205376
1        1  48.265476
2        2  39.312901
3        3  30.481630
4        4  39.188168
...    ...        ...
3499  3499  12.140996
3500  3500  24.600855
3501  3501  48.416721
3502  3502  41.432034
3503  3503  44.516960

[3504 rows x 2 columns]


In [10]:
submission.to_csv("submit_final.csv", index=False)